Look at effect on m5 as we transition through different versions of the throughput files.

In [1]:
import os
import shutil
import subprocess
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import rubin_sim.phot_utils as phot_utils
import syseng_throughputs as se

The survey simulations used 1.5 for simulations v1.5 through v3.2 (perhaps switching to 1.6 at some point, but this was a minor change). Then moved to v1.9 for v3.3.
Not all tags in the repo corresponded to meaningful changes in the throughput files (there are some tags which correspond to software or repository layout changes). 

In [2]:
defaultDirs = se.setDefaultDirs()
defaultDirs

{'detector': '/Users/lynnej/lsst_repos/syseng_throughputs/components/camera/detector/joint_minimum',
 'lens1': '/Users/lynnej/lsst_repos/syseng_throughputs/components/camera/lens1',
 'lens2': '/Users/lynnej/lsst_repos/syseng_throughputs/components/camera/lens2',
 'lens3': '/Users/lynnej/lsst_repos/syseng_throughputs/components/camera/lens3',
 'filters': '/Users/lynnej/lsst_repos/syseng_throughputs/components/camera/filters',
 'mirror1': '/Users/lynnej/lsst_repos/syseng_throughputs/components/telescope/mirror1',
 'mirror2': '/Users/lynnej/lsst_repos/syseng_throughputs/components/telescope/mirror2',
 'mirror3': '/Users/lynnej/lsst_repos/syseng_throughputs/components/telescope/mirror3',
 'atmosphere': '/Users/lynnej/lsst_repos/syseng_throughputs/siteProperties'}

In [3]:
# Read some common (and should not change over time) dark sky and atmospheric throughput files)
atmos = phot_utils.Bandpass()
atmos.read_throughput(os.path.join(defaultDirs['atmosphere'], 'atmos_10_aerosol.dat'))

darksky = phot_utils.Sed()
darksky.read_sed_flambda(os.path.join(defaultDirs['atmosphere'], 'darksky.dat'))

In [4]:
tags = ['1.3', '1.4', '1.5', '1.6', '1.7', '1.8', '1.9']

lsst_system = {}
lsst_hardware = {}
m5s = {}
for t in tags:
    gitcommand = f"git checkout tags/{t}"
    subprocess.run(args=gitcommand.split(), check=True)
    lsst_hardware[t], lsst_system[t] = se.buildHardwareAndSystem(defaultDirs, 
                                                                 atmosphereOverride=atmos, 
                                                                 addLosses=True)
    m5s[t] = se.makeM5(lsst_hardware[t], lsst_system[t], darksky=darksky, sky_mags=None,
           exptime={'u': 30, 'g': 15, 'r': 15, 'i': 15, 'z': 15, 'y': 15},
           nexp={'u': 1, 'g': 2, 'r': 2, 'i': 2, 'z': 2, 'y': 2},
           readnoise=8.8, othernoise=0, darkcurrent=0.2,
           effarea=np.pi*(6.423/2*100)**2, X=1.0, fwhm500=None)        

error: Your local changes to the following files would be overwritten by checkout:
	documentation/Components.ipynb
	documentation/Confirm Updates.ipynb
	documentation/read-lca18.ipynb
	notebooks/Components.ipynb
	notebooks/EvalReadnoise.ipynb
	notebooks/Mixed Detector Photometry.ipynb
	notebooks/Overview Paper.ipynb
	notebooks/SilverVsAluminum.ipynb
	notebooks/SilverVsAluminum_Stars.ipynb
	notebooks/Syseng Throughputs Repo Demo.ipynb
	notebooks/generate_sims_values.ipynb
	notebooks/m5_changes.ipynb
	syseng_throughputs/m5Utils.py
Please commit your changes or stash them before you switch branches.
Aborting


CalledProcessError: Command '['git', 'checkout', 'tags/1.3']' returned non-zero exit status 1.

In [ ]:
other = [] # git sha here if necessary
for t in other:
    gitcommand = f"git checkout {t}"
    subprocess.run(args=gitcommand.split(), check=True)
    lsst_hardware[t], lsst_system[t] = se.buildHardwareAndSystem(defaultDirs, 
                                                                 atmosphereOverride=atmos, 
                                                                 addLosses=True)
    m5s[t] = se.makeM5(lsst_hardware[t], lsst_system[t], darksky=darksky, sky_mags=None,
           exptime={'u': 30, 'g': 15, 'r': 15, 'i': 15, 'z': 15, 'y': 15},
           nexp={'u': 1, 'g': 2, 'r': 2, 'i': 2, 'z': 2, 'y': 2},
           readnoise=8.8, othernoise=0, darkcurrent=0.2,
           effarea=np.pi*(6.423/2*100)**2, X=1.0, fwhm500=None) 

In [ ]:
gitcommand = f"git checkout main"
subprocess.run(args=gitcommand.split(), check=True)

In [ ]:
# Just to track other input conditions
m5s['1.9']

In [ ]:
m5_srd_min = m5s['1.9']['m5_min']
m5_srd_fid = m5s['1.9']['m5_fid']

In [ ]:
m5 = pd.DataFrame([m5s[t]['m5'] for t in m5s] + [m5_srd_fid, m5_srd_min], index=[[f'm5 for {t}' for t in m5s] + ['m5 SRD design', 'm5 SRD minimum']])

In [ ]:
m5.round(2)

In [ ]:
colors =  {"u": "cyan", "g": "g", "r": "orange", "i": "r", "z": "m", "y": "b"}
versions = list(m5s.keys())
plt.figure(figsize=(8, 5))
for f in 'ugrizy':
    plt.plot(versions, m5[f][:-2], color=colors[f], marker='o', linestyle='-', label=f)
plt.legend(loc=(1.01, .3), fancybox=True, fontsize='x-large')
plt.grid(True, alpha=0.5)
plt.ylabel("Magnitude", fontsize='x-large')
plt.xlabel("Throughputs Version", fontsize='x-large')

In [ ]:
d = m5[4:7]
(d - d.loc['m5 for 1.7'].values).round(2)

In [ ]:
tt = ['1.7', '1.8', '1.9',]
colors =  {"u": "cyan", "g": "g", "r": "orange", "i": "r", "z": "m", "y": "b"}
linestyles = {'1.3': '-.', '1.4': '-.', '1.7': '-.', '1.8': ':', '1.9': '-'}

plt.figure(figsize=(8, 5))
for t in tt:
    for f in 'ugrizy':
        plt.plot(lsst_system[t][f].wavelen, lsst_system[t][f].sb, color=colors[f], linestyle=linestyles[t], label=f"{f} {t}")
plt.legend(loc=(1.01, 0.4), ncols=3, fancybox=True, fontsize='large')
plt.xlabel("Wavelength (nm)", fontsize='large')
plt.ylabel("Throughput", fontsize='large')
plt.ylim(0, 0.7)
plt.grid(True, alpha=0.5)

In [ ]:
# Effective time calculation
# Fractional change in effective time is proportional to change in m5.
# HERE - longer time = better
delta_m5 = (m5[:-2] - m5.loc['m5 for 1.7'].values)
dtime = np.power(10, 0.8 * (delta_m5))
dtime.index = [i[0].replace('m5 for', "Teff for") for i in dtime.index.values]
dtime[-3:]

In [ ]:
# SRD time distribution is approximate weight per filter (like the distribution of visits in the SRD)
SRD_time_distribution = pd.Series({'u': 0.068, 'g': 0.097, 'r': 0.223, 'i': 0.223, 'z': 0.194, 'y': 0.194})
SRD_time_distribution, SRD_time_distribution.sum()

In [ ]:
# So weight the difference in effective time by the fraction of visits in that filter -- look at the sum to see the overall effect.
weighted_dtime = dtime.multiply(SRD_time_distribution)
weighted_dtime['sum'] = weighted_dtime.sum(axis=1)
weighted_dtime.round(2)

In [ ]:
pd.DataFrame(weighted_dtime.loc[["Teff for 1.7", "Teff for 1.9"], 'sum']).round(2)